# 不等流計算3:一般断面の不等流計算2-分割断面法（平均流速公式レベル2）

## 課題
分割断面法のクラスを以下のように定義する。

※前回は追加距離をLと定義しましたがdistanceに変更しました。

In [1]:
import numpy as np

class subsection(object):
    def __init__(self, x, y, n):
        self.x = x
        self.y = y
        self.n = n

    def H2ABSKn(self, H): #変更:引数は水位のみ
        A, B, S, SN = float(0), float(0), float(0), float(0)
        x = self.x #変更:メンバ変数を参照
        y = self.y #変更:メンバ変数を参照
        n = self.n #変更:メンバ変数を参照
        
        for i in range(1, len(x)):
            dx = x[i] - x[i-1]    
            dy = y[i] - y[i-1]    
            hb, hf = H - y[i-1], H - y[i]
            
            if hb <= float(0) :
                if hf > float(0) :
                    dx_dh = dx / (hf - hb)
                    B += hf * dx_dh
                    A += 0.5 * hf * hf * dx_dh
                    Sp = hf * np.sqrt( dx_dh * dx_dh + 1.0)
                    S +=  Sp
                    SN += Sp * n[i-1]**1.5
            elif hf <= float(0) :
                if hb > float(0) :
                    dx_dh = dx / (hf - hb)
                    B -= hb * dx_dh
                    A -= 0.5 * hb * hb * dx_dh
                    Sp = hb * np.sqrt(dx_dh * dx_dh + 1.0)
                    S += Sp
                    SN += Sp * n[i-1]**1.5
            else :
                B += dx
                A += 0.5 * dx * (hf + hb)
                Sp = np.sqrt(dx**2 + dy**2)
                S += Sp
                SN += Sp * n[i-1]**1.5
                
        if S <= float(0):
            nd = float(0)
            K = float(0)
        else:
            nd = (SN/S)**(2.0/3.0)
            K = A**(5.0/3.0)/nd/S**(2.0/3.0)
            
        return A, B, S, K, nd
    
    
class section(object):
    def __init__(self, X, Y, N, distance=np.nan):
        self._subsections = [subsection(x,y,n) for x,y,n in zip(X,Y,N)]
        self.distance = distance
        
    def H2ABSKnSub(self, H):
        num = len(self._subsections)
        A = np.zeros(num)
        B = np.zeros(num)
        S = np.zeros(num)
        K = np.zeros(num)
        n = np.zeros(num)
            
        for nump in range(num) : 
            A[nump], B[nump], S[nump], K[nump], n[nump] = self._subsections[nump].H2ABSKn(H)
            
        return A, B, S, K, n
    
    def H2ABSK(self, H):
        A, B, S, K, n = self.H2ABSKnSub(H)
        return np.sum(A), np.sum(B), np.sum(S), np.sum(K)
    
    def zbmin(self):
        return np.array([s.y.min() for s in self._subsections]).min()
        

In [2]:
# 断面のデータ

x=np.array([0, 5,93,100,200,206,294,300])
y=np.array([6, 3.5, 3.5,  0,  0,  3,  3,  6])
n=np.array([0.041, 0.041,0.030,0.030,0.030,0.040,0.040])

X = [x[0:3], x[2:6], x[5:]]
Y = [y[0:3], y[2:6], y[5:]]
N = [n[0:2], n[2:5], n[5:]]

s = section(X,Y,N,0)
A,B,S,K = s.H2ABSK(5.0)
print('河積：{}\n水面幅：{}\n潤辺：{}\n通水能：{}'.format(A, B, S, K))

河積：858.0
水面幅：296.0
潤辺：298.3606797749979
通水能：62445.88519359645


### 課題1

 1. 新たに定義したクラスsectionに運動量補正係数、エネルギー補正係数、井田の合成径深を計算するメソッドを追加せよ。
 2. 新たに定義したクラスsectionに等流時水位、限界流時水位を計算するメソッドを追加せよ。

### 課題2

課題1で作成したクラスを用いて不等流計算モデルを実装し、縦断水位計算せよ。計算条件は以下のとおりとする。

 - 流量：$1500\rm{m^3/s}$
 - 下流端水位：等流水深（等流時水位）
 
断面データは前回課題と同様に以下のように定義する。

In [3]:
x=np.array([0, 5,93,100,200,206,294,300])
y=np.array([6, 3.5, 3.5,  0,  0,  3,  3,  6])
n=np.array([0.041, 0.041,0.030,0.030,0.030,0.040,0.040])

dx = 200
Ls = np.arange(0,15001,dx) 

sections = []
sect = {'x':x, 'y':y, 'n':n, 'L':Ls[0]}
sections.append(sect)

yold = y.copy()
ibs = []
ibs.append(1/2000)
for L in Ls[1:]:
    if L < 5000:
        ib = 1/2000
    elif L < 10000:
        ib = 1/1000
    else:
        ib = 1/500
        
    ynew = yold + dx*ib
    sect = {'x':x, 'y':ynew, 'n':n, 'L':L}
    sections.append(sect)
    yold = ynew.copy()
    ibs.append(ib)

In [4]:
csections = []

for s in sections:
    x, y, z, l = s['x'],s['y'],s['n'],s['L']
    X = [x[0:3], x[2:6], x[5:]]
    Y = [y[0:3], y[2:6], y[5:]]
    N = [n[0:2], n[2:5], n[5:]]
    csections.append( section(X,Y,N,l) )

In [5]:
# test
# for s in csections:
#     A,B,S,K = s.H2ABSK(s.zbmin()+5.0)
#     print(s.distance, A, B)

### 課題3

三重対角行列の数値解法であるTDMA法（トーマス法）について調べて、式を導出し、ソースコードを作成せよ。

### 課題4：アンケート

 - クラスの構成について意見をお願いします。無ければ大丈夫です。
     - メンバ変数について
         - 例えば、A,Bなどはメンバ変数にしたほうが良いか？私はプリミティブ変数（H）をクラスの外で定義してそれ以外の定数をメンバ変数として、クラスのメンバ変数は最小限とする書き方が好きです。
     - 変数、関数のネーミング
         - javaみたいにわかりやすく長くする。 x ⇒ coordsX、distance⇒distanceFromDownBoundaryなど
         - 関数名はどうする？H2ABSK ⇒ calsectionProfileとか
     - private,publicの関数、変数の明確化、getter、setterの定義等
     
    ...etc
     